In [9]:
from rediscluster import RedisCluster
import sys

def redis_cluster():
    redis_nodes =  [
                    {'host':'192.168.0.1','port':7000},
                    {'host':'192.168.0.2','port':7000},
                    {'host':'192.168.0.3','port':7000},
                   ]
    try:
        return RedisCluster(startup_nodes=redis_nodes, password= "REDIS_PASSWORD")
    except Exception as e:
        print("Connect Error!")
        sys.exit(1)

client = redis_cluster()
#     redis_conn.lpush('11', 1,2,3,4)
#     redis_conn.delete('11')
    
#     print("len:",redis_conn.llen('11'))

## Print Redeem Code - LLen

In [10]:
def print_length(client, key):
    print("[Codes Length]: %d" % client.llen(key))
    
    
print_code(client, '12')

[Code]: 9883


## Insert Redeem Code - LLen, LPush

In [15]:
def insert_redeem_code(client, key, codes):
    print("[Key]: %s" % key)
    print("[Before Insert Code Length]: %d" % client.llen(key))
    for code in codes:
        client.lpush(key, code)
    print("[Length]: %d" % len(codes))
    print("[After Insert Code Length]: %d" % client.llen(key))
    
    
insert_redeem_code(client, '02',['123'])

[Key]: 02
[Before Insert Code Length]: 995070
[Length]: 3
[After Insert Code Length]: 995073


In [18]:
# Print Data
redis_nodes =  [
                {'host':'192.168.0.1','port':7000},
                {'host':'192.168.0.2','port':7000},
                {'host':'192.168.0.3','port':7000},
                ]
try:
    redis_conn = RedisCluster(startup_nodes=redis_nodes,password= "REDIS_PASSWORD")
except Exception as e:
    print("Connect Error!")
    print(e)
    sys.exit(1)

print("'12'這個key的長度")
print("len:",redis_conn.llen('12'))
print("所有'12'這個的list")
print("all list data:",redis_conn.lrange('12',0,-1))

len: 75000
array [b'3', b'2', b'1', b'gcard_sn_39999', b'gcard_sn_39998', b'gcard_sn_39997', b'gcard_sn_39996', b'gcard_sn_39995', b'gcard_sn_39994', b'gcard_sn_39993', b'gcard_sn_39992', b'gcard_sn_39991', b'gcard_sn_39990', b'gcard_sn_39989', b'gcard_sn_39988', b'gcard_sn_39987', b'gcard_sn_39986', b'gcard_sn_39985', b'gcard_sn_39984', b'gcard_sn_39983', b'gcard_sn_39982', b'gcard_sn_39981', b'gcard_sn_39980', b'gcard_sn_39979', b'gcard_sn_39978', b'gcard_sn_39977', b'gcard_sn_39976', b'gcard_sn_39975', b'gcard_sn_39974', b'gcard_sn_39973', b'gcard_sn_39972', b'gcard_sn_39971', b'gcard_sn_39970', b'gcard_sn_39969', b'gcard_sn_39968', b'gcard_sn_39967', b'gcard_sn_39966', b'gcard_sn_39965', b'gcard_sn_39964', b'gcard_sn_39963', b'gcard_sn_39962', b'gcard_sn_39961', b'gcard_sn_39960', b'gcard_sn_39959', b'gcard_sn_39958', b'gcard_sn_39957', b'gcard_sn_39956', b'gcard_sn_39955', b'gcard_sn_39954', b'gcard_sn_39953', b'gcard_sn_39952', b'gcard_sn_39951', b'gcard_sn_39950', b'gcard_sn_399

In [ ]:
#     redisconn.set('name','admin')
#     redisconn.set('age',18)
#     print "age  is: ", redisconn.get('age')